In [7]:
import os
import requests
from bs4 import BeautifulSoup
import chromadb
import openai
from openai import OpenAI
from dotenv import load_dotenv

In [8]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [9]:
load_dotenv()

True

In [10]:
def cleaning(x):
  x=x.replace('\n',' ').replace('^',' ').replace('[edit]',' ')
  x=' '.join(x.split())
  return x

url = "https://en.wikipedia.org/wiki/Dentistry"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the content
content = soup.find('div', {'id': 'bodyContent'}).get_text()
paragraphs = content.split('\n\n')

new_cleaned_text=[]
for i in paragraphs:
  if i == '\nReferences[edit]':
    break
  if i == '':
    paragraphs.remove(i)
  else:
    i=cleaning(i)
    new_cleaned_text.append(i)
del(new_cleaned_text[0])
new_cleaned_text

['Branch of medicine',
 'This article contains too many pictures for its overall length. Relevant discussion may be found on the talk page. Please improve this article by removing indiscriminate collections of images or adjusting images that are sandwiching text in accordance with the Manual of Style on use of images. (Learn how and when to remove this message) DentistryA dentist treats a patient with the help of a dental assistant.OccupationNames Dentist Dental Surgeon Doctor [1][nb 1]',
 'Occupation typeProfessionActivity sectorsHealth care, Anatomy, Physiology, Pathology, Medicine, Pharmacology, SurgeryDescriptionCompetencies Sub-Millimeter Surgical Dexterity Knowledge of human health, disease, pathology, and anatomy Communication/Interpersonal Skills Analytical Skills Critical Thinking Empathy/Professionalism Education requiredDental DegreeFields ofemploymentPrivate practicesPrimary care clinicsHospitalsRelated jobsPhysiciandental assistantdental techniciandental hygienistvarious d

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
HuggingFaceEmbeddings_ = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/home/zigron/anaconda3/envs/codeenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/zigron/anaconda3/envs/codeenv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [12]:
from langchain_community.vectorstores import Chroma
chroma_vector_database = Chroma.from_texts(texts=new_cleaned_text, embedding=HuggingFaceEmbeddings_)
retriever = chroma_vector_database.as_retriever(search_kwargs={"k":5})

In [13]:
# from langchain.memory import ChatMessageHistory


In [14]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

In [15]:
from langchain.prompts import PromptTemplate

In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [17]:
from langchain_core.messages import AIMessage, HumanMessage

In [18]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [19]:
from langchain.memory import ConversationBufferMemory


In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

In [22]:
from langchain.memory import ChatMessageHistory

In [23]:
from langchain.memory import ChatMessageHistory

In [24]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)

In [28]:

demo_ephemeral_chat_history = ChatMessageHistory()
while True:

    query=input('Enter your query: ')
    if query=='exit':
        break
    top_doc=retriever.invoke(query)
    SYSTEM_TEMPLATE ="""
    Act as a dentist. Whenever someone will ask you about dentistry, provide the specific and to the point information. you dont have to create long passages. If you don't know the answer or it is irrelevant to dentisty, just say that you don't know or this is out of my scope ask me about desntistry related queries, don't try to make up an answer.
    you also have to same the previous responses you generated. If use ask you something about generated responses, you have to assist them.
    you have to behave like an assistant.
    {context}
    """

    question_answering_prompt = ChatPromptTemplate.from_messages(
        [
            (
               "system",
            SYSTEM_TEMPLATE,
            ),
        MessagesPlaceholder(variable_name="messages"),
        ]
    )

    demo_ephemeral_chat_history.add_user_message(query)

    # # system = [{'role':'system','content':SYSTEM_TEMPLATE}]
    # system=[]
    
    document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

    res = document_chain.invoke(
        {
            "messages": demo_ephemeral_chat_history.messages,
            "context": top_doc,
        }   
    )

    demo_ephemeral_chat_history.add_ai_message(res)
    print(f"Assistant: {res}")
    # prev_history.append([HumanMessage(content=query)])
    # prev_history.append(res)
    # system.extend(prev_history)

    # print(res)

Assistant: I am here to provide information and answer any questions you may have about dentistry. How can I assist you today?
Assistant: - Complete a bachelor's degree
- Attend a dental school for 4 years to earn a Doctor of Dental Surgery (DDS) or Doctor of Dental Medicine (DMD) degree
- Consider specializing in areas like Orthodontics, Endodontics, or Oral Surgery
- Obtain necessary licensing and certifications to practice as a dentist


In [29]:
demo_ephemeral_chat_history.messages

[HumanMessage(content='who are you?'),
 AIMessage(content='I am here to provide information and answer any questions you may have about dentistry. How can I assist you today?'),
 HumanMessage(content='how can i become a dentist. tell me in bullets'),
 AIMessage(content="- Complete a bachelor's degree\n- Attend a dental school for 4 years to earn a Doctor of Dental Surgery (DDS) or Doctor of Dental Medicine (DMD) degree\n- Consider specializing in areas like Orthodontics, Endodontics, or Oral Surgery\n- Obtain necessary licensing and certifications to practice as a dentist")]

In [27]:

    # prompt=PromptTemplate(
    #     input_variables=[{'context':"chat_history",'question':'query'}],template=template_
    # )
    # print(prompt)
    # memory=ConversationBufferMemory(memory_key='chat_history')
    # llm_chain=LLMChain(
    #     llm=llm,
    #     prompt=prompt,
    #     memory=memory,
    # )

    # llm_chain.invoke(query)
    # prompt = ChatPromptTemplate.from_messages(
    #     [
    #         (
    #             "system",
    #             template_,
    #         ),
    #      MessagesPlaceholder(variable_name="chat_history"),("human", "{query}"),
    #     ]
    # )

    # chain = prompt | llm

    # chain_with_message_history = RunnableWithMessageHistory(
    # chain,
    # lambda session_id: demo_ephemeral_chat_history,input_messages_key="input",history_messages_key="chat_history",
    # )

    # chain_with_message_history.revoke(
    #     {"input": "What's my name?"},
    #     {"configurable": {"session_id": "unused"}},
    #)


    # prompt = ChatPromptTemplate.from_messages(
    #     [
    #         SystemMessage(
    #            content=template_
    #         ),
    #         MessagesPlaceholder(
    #            variable_name="chat_history"
    #      ),def get_completion(prompt, model=llm_model):
    # messages = [{"role": "user", "content": prompt}]
    # response = openai.ChatCompletion.create(
    #     model=model,
    #     messages=messages,
    #     temperature=0, 
    # )
    # return response.choices[0].message["content"]

    #      ),  
    #     ]
    # ) 

    # memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # chain = LLMChain(
    # llm=llm,
    # prompt=prompt,
    # verbose=True,
    # memory=memory,
    # )

    # answer = chain.invoke(query)
    # print(answer['text'])

    # query_answer = chain.invoke({"context":top_doc, "question":query})

    # ai_message=query_answer.content


    # history.add_user_message(query)

    # history.add_ai_message(ai_message)


    # print('Your query: ',query)
    # print('Chatbot response: ',ai_message)
    # print(history)